# **Práctica Final por Equipo, Propedéutico de Estadística Verano 2020**
____________________________________________________________________________________________________________

### **Análisis descriptivo de los Taxis Verdes de la Ciudad de Nueva York durante 2019 en R, utilizando la paquetería de tidyverse y R base y las librerías de readr, ggplot2, modeest**


### **Profr.** Erick Palacios Moreno
### **Equipo 8**
### **Integrantes**
#### Carlos Garza (carlosgarza-3)
#### Enrique Ortiz Casillas (EnriqueOrtiz27)
#### Octavio Fuentes Ortiz (ofuentesitam)
#### **Fecha de Entrega:** 19/07/2020
____________________________________________________________________________________________________________

**La base de datos de los Taxis Verdes, la obtuvimos de la página del Gobierno de la Ciudad de Nueva York.**

**La base de datos se puede consultar en:**  *[Green Taxi Trip Records NYC](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page)* **.**

**El diccionario de datos se puede consultar en:**  *[Green Trips Data Dictionary](https://www1.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_green.pdf)* **.**
____________________________________________________________________________________________________________

**Paqueterías utilizadas:**

- *[tidyverse](https://www.tidyverse.org/)*.

In [1]:
#install.packages("tidyverse")
library(tidyverse) #ggplot2,tibble,tidyr,readr,purrr,dplyr,stringr,forcats

Warning message:
"package 'tidyverse' was built under R version 3.6.3"
-- Attaching packages --------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.2     v purrr   0.3.4
v tibble  3.0.2     v dplyr   1.0.0
v tidyr   1.1.0     v stringr 1.4.0
v readr   1.3.1     v forcats 0.5.0

Warning message:
"package 'ggplot2' was built under R version 3.6.3"
Warning message:
"package 'tibble' was built under R version 3.6.3"
Warning message:
"package 'tidyr' was built under R version 3.6.3"
Warning message:
"package 'readr' was built under R version 3.6.3"
Warning message:
"package 'purrr' was built under R version 3.6.3"
Warning message:
"package 'dplyr' was built under R version 3.6.3"
Warning message:
"package 'stringr' was built under R version 3.6.3"
Warning message:
"package 'forcats' was built under R version 3.6.3"
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



- *[modeest](https://www.rdocumentation.org/packages/modeest/versions/2.4.0#:~:text=modeest%3A%20Estimation%20of%20the%20mode,modes%20of%20usual%20probability%20distributions.)*.

In [2]:
#install.packages("modeest")
library(modeest)

Warning message:
"package 'modeest' was built under R version 3.6.3"
Registered S3 method overwritten by 'rmutil':
  method         from
  print.response httr



- *[R base](https://rstudio.com/wp-content/uploads/2016/05/base-r.pdf)*.

**Librerías utilizadas:**

- *[readr](https://readr.tidyverse.org/)*.

In [3]:
library(readr)

- *[ggplot2](https://ggplot2.tidyverse.org/)*.

In [4]:
library(ggplot2)

____________________________________________________________________________________________________________

Establecemos el directorio donde tenemos nuestras BBDD:

In [82]:
setwd("C:/Users/ofuentes/Desktop/green_taxi_records")

____________________________________________________________________________________________________________

#### ___LIMPIEZA DE DATOS___ 

Antes de empezar con el análisis descriptivo de nuestros datos es necesario saber con qué datos estamos trabajando y sobretodo, si éstos están limpios. El procedimiento y explicación de nuestra limpieza de datos, se encuentra en el repositorio de nuestra Práctica Final: [data_exploration.ipynb](https://github.com/prope-2020-gh-classroom/practica-final-por-equipos-verano-2020-itam-carlosgarza-3/blob/master/tercer_cuatrimestre/data_exploration.ipynb).

Nuestras funciones de limpieza y adición de columnas, se resumen en la función **"data_pipeline"**:

In [84]:
clean_dataframe <- function(data_frame) {
    #function to remove unnecessary cols in r dataframe and na's
    data_frame$VendorID <- NULL
    data_frame$store_and_fwd_flag <- NULL
    data_frame$PULocationID <- NULL
    data_frame$DOLocationID <- NULL
    data_frame$RatecodeID <- NULL
    data_frame$ehail_fee <- NULL
    data_frame$improvement_surcharge <- NULL 
    data_frame$tolls_amount <- NULL 
    data_frame$mta_tax <- NULL 
    data_frame$extra <- NULL 
    data_frame$congestion_surcharge <- NULL
    return(data_frame[complete.cases(data_frame),])
}


add_columns <- function(data_frame, month) {
    #function to add 2 columns to data frame to improve analysis
    #first we add trip duration in hours
    data_frame$lpep_dropoff_datetime <- as.POSIXct(data_frame$lpep_dropoff_datetime,
                                      format='%Y-%m-%d %H:%M:%S')
    data_frame$lpep_pickup_datetime <- as.POSIXct(data_frame$lpep_pickup_datetime,
                                      format='%Y-%m-%d %H:%M:%S')
    data_frame$trip_duration <- as.numeric(difftime(data_frame[,2], data_frame[,1], units="hours"))
    
    #then we add tip percentage
    data_frame$tip_pct <- data_frame$tip_amount / data_frame$total_amount
    data_frame$month <- month
    data_frame <- data_frame[data_frame$trip_duration<5,]
    data_frame <- data_frame[data_frame$total_amount > 0,]
    data_frame <- data_frame[data_frame$fare_amount > 0,]
    data_frame <- data_frame[data_frame$tip_amount > 0,]
    data_frame <- data_frame[data_frame$trip_duration >0,]
    data_frame <- data_frame[data_frame$trip_distance >0,]
    return(data_frame)
}


data_pipeline <- function(csv, month){
    #function to include all cleaning and transformation functions 
    data <- read.csv(csv)
    data <- clean_dataframe(data)
    data <- add_columns(data, month)
    return(data)
}

Aplicamos **data_pipeline** para el **primer cuatrimestre:**

In [85]:
enero_data <- data_pipeline('green_tripdata_2019-01.csv', 'Enero')
febrero_data <- data_pipeline('green_tripdata_2019-02.csv', 'Febrero')
marzo_data <- data_pipeline('green_tripdata_2019-03.csv', 'Marzo')
abril_data <- data_pipeline('green_tripdata_2019-04.csv', 'Abril')

Juntamos nuestro **primer cuatrimestre** y lo exportamos a un archivo .csv (para posteriormente leerlo):

In [86]:
cuatrimestre1_data <- bind_rows(enero_data, febrero_data, marzo_data, abril_data)
write.csv(x = cuatrimestre1_data, file = "primer_cuatri.csv")
primer_cuatrimestre <- read_csv('primer_cuatri.csv')
# Colocamos nombre a las columnas:
colnames(primer_cuatrimestre) <- c("X1","Pick_up","Dropp_off","Passengers","Trip_distance","Fare_amount",
                                   "Tip_amount", "Total_amount","Payment_type", "Trip_type", "Trip_duration",
                                   "Tip_pct", "Month")

Warning message:
"Missing column names filled in: 'X1' [1]"
Parsed with column specification:
cols(
  X1 = col_character(),
  lpep_pickup_datetime = col_datetime(format = ""),
  lpep_dropoff_datetime = col_datetime(format = ""),
  passenger_count = col_double(),
  trip_distance = col_double(),
  fare_amount = col_double(),
  tip_amount = col_double(),
  total_amount = col_double(),
  payment_type = col_double(),
  trip_type = col_double(),
  trip_duration = col_double(),
  tip_pct = col_double(),
  month = col_character()
)



Vemos la estructura de la base de datos ya limpia:

In [87]:
str(primer_cuatrimestre)

tibble [802,074 x 13] (S3: spec_tbl_df/tbl_df/tbl/data.frame)
 $ X1           : chr [1:802074] "...1" "...2" "...3" "...4" ...
 $ Pick_up      : POSIXct[1:802074], format: "2019-01-01 00:46:20" "2019-01-01 00:37:00" ...
 $ Dropp_off    : POSIXct[1:802074], format: "2019-01-01 01:04:54" "2019-01-01 00:56:42" ...
 $ Passengers   : num [1:802074] 2 1 2 1 1 1 1 1 1 5 ...
 $ Trip_distance: num [1:802074] 2.68 6.84 1.15 1.2 5.5 5.01 0.43 2.72 2.15 2.18 ...
 $ Fare_amount  : num [1:802074] 13.5 22 6.5 7.5 19.5 20 4 10.5 10.5 9.5 ...
 $ Tip_amount   : num [1:802074] 2.96 8 1.56 5.2 4.15 4.26 1.06 2.36 2.95 3.24 ...
 $ Total_amount : num [1:802074] 19.71 37.06 9.36 14 24.95 ...
 $ Payment_type : num [1:802074] 1 1 1 1 1 1 1 1 1 1 ...
 $ Trip_type    : num [1:802074] 1 1 1 1 1 1 1 1 1 1 ...
 $ Trip_duration: num [1:802074] 0.309 0.328 0.12 0.147 0.351 ...
 $ Tip_pct      : num [1:802074] 0.15 0.216 0.167 0.371 0.166 ...
 $ Month        : chr [1:802074] "Enero" "Enero" "Enero" "Enero" ...
 - attr

Vemos un resumen de cada una de las columnas de la base de datos antes de empezar:

In [110]:
summary(primer_cuatrimestre)

      X1               Pick_up                      Dropp_off                  
 Length:802074      Min.   :2009-01-01 00:12:45   Min.   :2009-01-01 00:22:56  
 Class :character   1st Qu.:2019-01-30 14:47:39   1st Qu.:2019-01-30 15:01:48  
 Mode  :character   Median :2019-02-28 14:12:57   Median :2019-02-28 14:26:24  
                    Mean   :2019-03-01 07:28:14   Mean   :2019-03-01 07:41:09  
                    3rd Qu.:2019-03-31 08:32:16   3rd Qu.:2019-03-31 08:42:11  
                    Max.   :2019-06-15 04:36:12   Max.   :2019-06-15 04:44:15  
                    NA's   :703                   NA's   :703                  
   Passengers    Trip_distance      Fare_amount       Tip_amount     
 Min.   :0.000   Min.   :  0.010   Min.   :  0.01   Min.   :  0.010  
 1st Qu.:1.000   1st Qu.:  1.100   1st Qu.:  6.50   1st Qu.:  1.460  
 Median :1.000   Median :  1.860   Median :  9.50   Median :  2.060  
 Mean   :1.326   Mean   :  2.772   Mean   : 11.93   Mean   :  2.652  
 3rd Qu.:1

____________________________________________________________________________________________________________

#### ___PRIMER CUATRIMESTRE 2019___ 

- Nuestros datos se ven así:

In [10]:
# Enero
head(primer_cuatrimestre, n = 1)
# Abril
tail(primer_cuatrimestre, n = 1)

X1,Pick_up,Dropp_off,Passengers,Trip_distance,Fare_amount,Tip_amount,Total_amount,Payment_type,Trip_type,Trip_duration,Tip_pct,Month
<chr>,<dttm>,<dttm>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
...1,2019-01-01 00:46:20,2019-01-01 01:04:54,2,2.68,13.5,2.96,19.71,1,1,0.3094444,0.1501776,Enero


X1,Pick_up,Dropp_off,Passengers,Trip_distance,Fare_amount,Tip_amount,Total_amount,Payment_type,Trip_type,Trip_duration,Tip_pct,Month
<chr>,<dttm>,<dttm>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
514390,2019-04-30 23:40:13,2019-04-30 23:46:50,1,1.02,6.5,1.56,9.36,1,1,0.1102778,0.1666667,Abril


**Medidas de Tendencia Central** :

- Medias del **primer cuatrimestre**

In [11]:
# Media de pasajeros
passengers_mean <-as.numeric(format(round(mean(primer_cuatrimestre$Passengers, na.rm = TRUE), 2), nsmall = 2))
# Media de la distancia del viaje
trip_mean <- as.numeric(format(round(mean(primer_cuatrimestre$Trip_distance, na.rm = TRUE), 2), nsmall = 2))
# Media de la tarifa
fare_mean <- as.numeric(format(round(mean(primer_cuatrimestre$Fare_amount, na.rm = TRUE), 2), nsmall = 2))
# Media de la propina
tip_mean <- as.numeric(format(round(mean(primer_cuatrimestre$Tip_amount, na.rm = TRUE), 2), nsmall = 2))
# Media del costo total del viaje
total_mean <- as.numeric(format(round(mean(primer_cuatrimestre$Total_amount, na.rm = TRUE), 2), nsmall = 2))
# Media de la duración del viaje
duration_mean <- as.numeric(format(round(mean(primer_cuatrimestre$Trip_duration, na.rm = TRUE), 2), nsmall = 2))

In [12]:
columnas <- c("Media Pasajeros", "Media Distancia", "Media Tarifa", "Media Propina", "Media Costo Viaje",
             "Media Duración")
valores <- c(passengers_mean, trip_mean, fare_mean, tip_mean, total_mean, duration_mean)
medias_cuatrimestre <- data.frame(columnas, valores)

- Medianas del **primer cuatrimestre**

In [13]:
# Mediana de pasajeros
passengers_median <-as.numeric(format(round(median(primer_cuatrimestre$Passengers, na.rm = TRUE), 2), nsmall = 2))
# Mediana de la distancia del viaje
trip_median <- as.numeric(format(round(median(primer_cuatrimestre$Trip_distance, na.rm = TRUE), 2), nsmall = 2))
# Mediana de la tarifa
fare_median <- as.numeric(format(round(median(primer_cuatrimestre$Fare_amount, na.rm = TRUE), 2), nsmall = 2))
# Mediana de la propina
tip_median <- as.numeric(format(round(median(primer_cuatrimestre$Tip_amount, na.rm = TRUE), 2), nsmall = 2))
# Mediana del costo total del viaje
total_median <- as.numeric(format(round(median(primer_cuatrimestre$Total_amount, na.rm = TRUE), 2), nsmall = 2))
# Mediana de la duración del viaje
duration_median <- as.numeric(format(round(median(primer_cuatrimestre$Trip_duration, na.rm = TRUE), 2), nsmall = 2))

In [14]:
columnas_median <- c("Mediana Pasajeros", "Mediana Distancia", "Mediana Tarifa", "Mediana Propina", 
                     "Mediana Costo Viaje", "Mediana Duración")
valores_median <- c(passengers_median, trip_median, fare_median, tip_median, total_median, duration_median)
medianas_cuatrimestre <- data.frame(columnas_median, valores_median)

- Modas del **primer cuatrimestre**

In [15]:
# Moda de pasajeros
passengers_mode <-as.numeric(format(round(mfv(primer_cuatrimestre$Passengers, na_rm = TRUE), 2), nsmall = 2))
# Moda de la distancia del viaje
trip_mode <- as.numeric(format(round(mfv(primer_cuatrimestre$Trip_distance, na_rm = TRUE), 2), nsmall = 2))
# Moda de la tarifa
fare_mode <- as.numeric(format(round(mfv(primer_cuatrimestre$Fare_amount, na_rm = TRUE), 2), nsmall = 2))
# Moda de la propina
tip_mode <- as.numeric(format(round(mfv(primer_cuatrimestre$Tip_amount, na_rm = TRUE), 2), nsmall = 2)) 
# Moda del costo total del viaje
total_mode <- as.numeric(format(round(mfv(primer_cuatrimestre$Total_amount, na_rm = TRUE), 2), nsmall = 2)) 
# Moda de la duración del viaje
duration_mode <- as.numeric(format(round(mfv(primer_cuatrimestre$Trip_duration, na_rm = TRUE)), 2), nsmall = 2)

In [16]:
columnas_mode <- c("Moda Pasajeros", "Moda Distancia", "Moda Tarifa", "Moda Propina", "Moda Costo Viaje", 
                   "Moda Duración")
valores_mode <- c(passengers_mode, trip_mode, fare_mode, tip_mode, total_mode, duration_mode)
modas_cuatrimestre <- data.frame(columnas_mode, valores_mode)

**Las medias quedan así:**

In [88]:
print('Medias de todo el primer cuatrimestre')
medias_cuatrimestre

[1] "Medias de todo el primer cuatrimestre"


columnas,valores
<fct>,<dbl>
Media Pasajeros,1.33
Media Distancia,2.75
Media Tarifa,11.94
Media Propina,2.66
Media Costo Viaje,16.46
Media Duración,0.21


**Las medianas quedan así:**

In [89]:
print('Medianas de todo el primer cuatrimestre')
medianas_cuatrimestre

[1] "Medianas de todo el primer cuatrimestre"


columnas_median,valores_median
<fct>,<dbl>
Mediana Pasajeros,1.00
Mediana Distancia,1.83
Mediana Tarifa,9.50
Mediana Propina,2.06
Mediana Costo Viaje,12.96
Mediana Duración,0.17


**Las modas quedan así:**

In [90]:
print('Modas de todo el primer cuatrimestre')
modas_cuatrimestre

[1] "Modas de todo el primer cuatrimestre"


columnas_mode,valores_mode
<fct>,<dbl>
Moda Pasajeros,1.00
Moda Distancia,0.90
Moda Tarifa,6.00
Moda Propina,1.00
Moda Costo Viaje,8.76
Moda Duración,0.00


**Medidas de Posición** :

- Frecuencias absolutas del **primer cuatrimestre** 

In [94]:
# Frecuencia absoluta del tipo de pago
pago_fa <- table(primer_cuatrimestre$Payment_type)
print('Frecuencia absoluta del tipo de pago')
pago_fa
# Frecuencia absoluta del tipo de viaje
viaje_fa <- table(primer_cuatrimestre$Trip_type)
print('Frecuencia absoluta del tipo de viaje')
viaje_fa
# Frecuencia absoluta del número de pasajeros
pasajeros_fa <- table(primer_cuatrimestre$Passengers)
print('Frecuencia absoluta del número de pasajeros')
pasajeros_fa

[1] "Frecuencia absoluta del tipo de pago"



     1      2      3      4 
801321     39      7      4 

[1] "Frecuencia absoluta del tipo de viaje"



     1      2 
796412   4959 

[1] "Frecuencia absoluta del número de pasajeros"



     0      1      2      3      4      5      6      7      8      9 
  1759 687950  58372   8970   3711  27426  13176      1      5      1 

- Frecuencias relativas del **primer cuatrimestre** 

In [95]:
# Frecuencia relativa del tipo de pago
pago_fr <- prop.table(pago_fa)*100
print('Frecuencia relativa del tipo de pago')
round(pago_fr,4)
# Frecuencia relativa del tipo de viaje
viaje_fr <- prop.table(viaje_fa)*100
print('Frecuencia relativa del tipo de viaje')
round(viaje_fr,4)
# Frecuencia relativa del número de pasajeros
pasajeros_fr <- prop.table(pasajeros_fa)*100
print('Frecuencia relativa del número de pasajeros')
round(pasajeros_fr,4)

[1] "Frecuencia relativa del tipo de pago"



      1       2       3       4 
99.9938  0.0049  0.0009  0.0005 

[1] "Frecuencia relativa del tipo de viaje"



      1       2 
99.3812  0.6188 

[1] "Frecuencia relativa del número de pasajeros"



      0       1       2       3       4       5       6       7       8       9 
 0.2195 85.8466  7.2840  1.1193  0.4631  3.4224  1.6442  0.0001  0.0006  0.0001 

**Medidas de Dispersión** :

- Rangos, mínimos y máximos del **primer cuatrimestre**

In [101]:
# Pasajeros
passengers_r <-range(primer_cuatrimestre$Passengers, na.rm = TRUE)
passengers_min <-min(primer_cuatrimestre$Passengers, na.rm = TRUE)
passengers_max <-max(primer_cuatrimestre$Passengers, na.rm = TRUE)
# Distancia del viaje
distance_r <- range(primer_cuatrimestre$Trip_distance, na.rm = TRUE)
distance_min <- min(primer_cuatrimestre$Trip_distance, na.rm = TRUE)
distance_max <- max(primer_cuatrimestre$Trip_distance, na.rm = TRUE)
# Tarifa del viaje
fare_r <- range(primer_cuatrimestre$Fare_amount, na.rm = TRUE)
fare_r <- min(primer_cuatrimestre$Fare_amount, na.rm = TRUE)
fare_r <- max(primer_cuatrimestre$Fare_amount, na.rm = TRUE)
# Monto de la propina
tip_r <- range(primer_cuatrimestre$Tip_amount, na.rm = TRUE)
tip_min <- min(primer_cuatrimestre$Tip_amount, na.rm = TRUE)
tip_max <- max(primer_cuatrimestre$Tip_amount, na.rm = TRUE)
# Monto total
total_r <- range(primer_cuatrimestre$Total_amount, na.rm = TRUE)
total_min <- min(primer_cuatrimestre$Total_amount, na.rm = TRUE)
total_max <- max(primer_cuatrimestre$Total_amount, na.rm = TRUE)
# Duración del viaje
duration_r <- range(primer_cuatrimestre$Trip_duration, na.rm = TRUE)
duration_min <- min(primer_cuatrimestre$Trip_duration, na.rm = TRUE)
duration_max <- max(primer_cuatrimestre$Trip_duration, na.rm = TRUE)

In [105]:
columnas_range <- c("Rango Pasajeros", "Rango Distancia", "Rango Tarifa", "Rango Propina", "Rango Costo Viaje", 
                   "Rango Duración")
valores_range <- c(passengers_r, distance_r, fare_r, tip_r, total_r, duration_r)

In [106]:
valores_range

[1] 0.000000e+00 9.000000e+00 1.000000e-02 6.666000e+02 3.000000e+02
 [6] 1.000000e-02 4.400000e+02 9.000000e-02 4.790500e+02 2.777778e-04
[11] 4.988056e+00

____________________________________________________________________________________________________________
#### ___SEGUNDO CUATRIMESTRE 2019___ 

____________________________________________________________________________________________________________
#### ___TERCER CUATRIMESTRE 2019___ 